#### Practice
#####  1. Write a decorator that ensures a function is only called by users with a specific role. Each function should have an user_type with a string type in kwargs

Example

@is_admin

def show_customer_receipt(user_type: str):
    # some very dangerous operation


show_customer_receipt(user_type='user')
> ValueError: Permission denied


show_customer_receipt(user_type='admin')
> function pass as it should be


In [29]:
def is_admin(func):       
    def wrapper(**kwargs):
        if kwargs['user_type']=='admin':
            return func(**kwargs)
        else:
            return 'ValueError: Permission denied'       
    return wrapper

In [30]:
@is_admin
def show_customer_receipt(user_type: str):
    return 'Yes'

In [31]:
show_customer_receipt(user_type='user')

'ValueError: Permission denied'

#####  2. Write a decorator that wraps a function in a try-except block and print an error if error has happened

Example

@catch_errors

def some_function_with_risky_operation(data):
    print(data['key'])


some_function_with_risky_operation({'foo': 'bar'})

> Found 1 error during execution of your function: KeyError no such key as foo

some_function_with_risky_operation({'key': 'bar'})

> bar


In [408]:

def catch_errors(func):
    message='Found 1 error during execution of your function: {0} -> {1}'
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except KeyError as k:
            print(message.format(type(k).__name__, 'No such key!'))
        except Exception as e:
            print(message.format(type(e).__name__, e))
                   
    return wrapper

In [409]:
@catch_errors
def some_function_with_risky_operation(data):
    return data['key']

In [411]:
some_function_with_risky_operation({'key':'bar'})

'bar'

#####  3. Optional: Create a decorator that will check types. It should take a function with arguments and validate inputs with annotations.

Example:

@check_types

def add(a: int, b: int) -> int:
    return a + b

add(1, 2)
> 3

add("1", "2")
> TypeError: Argument a must be int, not str

In [35]:
import inspect

In [378]:
def check_types(func):
    dict_annot = func. __annotations__
    
    def wrapper(*args, **kwargs): 
        # create dictionary of types of inputed variables and their names 
        values=inspect.signature(func).bind(*args, **kwargs)
        err_message='{0} must be {1}, not {2}'
        
        for name, value in values.arguments.items():
            if name in dict_annot.keys():
                if isinstance(value, dict_annot[name])==False:
                    raise TypeError(err_message.format(name, dict_annot[name].__name__, type(value).__name__))
                    
        if isinstance(func(*args, **kwargs), dict_annot['return'])==False:
            raise TypeError(err_message.format( func.__name__, dict_annot['return'].__name__ ,\
                                               type(func(*args, **kwargs)).__name__))
            
        return func(*args, **kwargs)
    return wrapper

In [379]:
@check_types
def func(num:int, txt:str='txt')->int:
    return num

In [380]:
func(1, 'ppp')

1

#####  4. Optional: Create a function that caches the result of a function, so that if it is called with same same argument multiple times, it returns the cached result first instead of re-executing the function. It`s one of the real task on the project

In [45]:
import os.path #module to check is file or not

def cache_result(func):
    file_cache='cache_func.csv' # file name to cache result of functions
    
    if os.path.exists(file_cache)==False: #if file not exist, create file with header
        with open(file_cache, mode='w', encoding='utf8') as file:
            file.write('Function_*args**kwargs;Return\n')
            
    def wrapper(*args, **kwargs):
        key =func.__name__ + str(args) + str(kwargs) #create key as name of function + its arguments
        cache={}
        with open(file_cache, mode='r', encoding='utf8') as file:
            # dictionary of cache
            header=file.readline()
            for row in file.readlines():
                name,value=row.strip().split(';')
                cache[name]=value
        if key not in cache:
            cache[key] = func(*args, **kwargs)
            with open(file_cache, mode='a', encoding='utf8') as file:
                file.write(f"{key};{cache[key]}\n")
        return cache[key]
    
    return wrapper


@cache_result
def hii(*args):
    return 'hello'
    
@cache_result
def create_list():
    _list = []
    for i in range(0, 20):
        _list.append(i)
    return _list

In [46]:
create_list()
create_list()
hii('hhh')
hii('hhhg')

'hello'

##### 5. Optional: Write a decorator that adds a rate-limiter to a function, so that it can only be called a certain amount of times per minute.

In [326]:
import time
def rate_limiter(max_call:int=3, limit_sec:int=60):
    times_call=[]
    def decor(func):
        def wrapper(*args, **kwargs):
            times_call.append(time.time())
            # difference between first  and last attempts
            diff_sec=times_call[-1]-times_call[0] 
            # if diff more than limit, start again
            if diff_sec>limit_sec:
                del times_call[-2::-1]
                diff_sec=0
            # check amount of attempts in limit period
            if len(times_call)>max_call and diff_sec<=limit_sec:                
                return f'Time is not over. Try again later'            
            return func(*args, **kwargs) 
        return wrapper
    return decor

In [327]:
@rate_limiter(max_call=3, limit_sec=60)
def func(num:int, txt:str='txt')->int:
    return num

In [328]:
print(func(1,txt='fghhff')) 

1
